# 1. Import dependencies

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime, timedelta

# 2. Scraping fraud-related news

In [3]:
base_url = "https://web-data.api.hk01.com/v2/issues/10221/relatedBlock/0/"

headers = {
    'Referer': 'https://www.hk01.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36'
}

params = {
    'limit': 6,
    'offset': 0,  # 初始 offset
    'bucketId': '00000'
}

# save all the articles
all_articles = []

# 记录循环次数，防止无限循环
request_count = 0
max_requests = 1000  # 设置一个上限，可以根据需要调整

print("开始获取文章列表...")
while True:
    print(f"正在请求第 {request_count + 1} 次 API...")
    
    # 发送 GET 请求
    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # 提取 items
        items = data.get('items', [])
        next_offset = data.get('nextOffset', None)
        
        print(f"  - 获取到 {len(items)} 篇文章")
        
        # 将本次获取的文章添加到总列表
        all_articles.extend(items)
        
        # 检查是否有 nextOffset
        if next_offset is not None:
            # 更新 params 中的 offset 为 nextOffset
            params['offset'] = next_offset
            print(f"  - 更新 offset 为: {next_offset}")
        else:
            # 如果没有 nextOffset，说明数据已加载完毕
            print("  - 没有更多数据，停止请求。")
            break
        
        # 检查是否达到最大请求数
        request_count += 1
        if request_count >= max_requests:
            print(f"  - 达到最大请求数 {max_requests}，停止请求。")
            break
        
        # 添加短暂延迟，避免请求过于频繁
        time.sleep(0.5)
        
    else:
        print(f"  - 请求失败，状态码: {response.status_code}")
        print(response.text)
        break

print(f"\n--- 总共获取到 {len(all_articles)} 篇文章列表 ---\n")

开始获取文章列表...
正在请求第 1 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769847711
正在请求第 2 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769684902
正在请求第 3 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769592948
正在请求第 4 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769487792
正在请求第 5 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769335233
正在请求第 6 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769153928
正在请求第 7 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769072210
正在请求第 8 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1768799694
正在请求第 9 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1768563674
正在请求第 10 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1768469664
正在请求第 11 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1768367022
正在请求第 12 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1768271307
正在请求第 13 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1767974334
正在请求第 14 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1767858718
正在请求第 15 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1767610960
正在请求第 16 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1767365159
正在请求第 17 次 API...
  - 获取到 6 篇文章
  - 更

# 3. Save the article to a Markdown document

In [4]:
output_filename = "hk01_scam_articles.md"
print(f"开始写入 Markdown 文件: {output_filename}")

with open(output_filename, 'w', encoding='utf-8') as f:
    f.write("# HK01 騙案相關文章\n\n")
    f.write(f"共 {len(all_articles)} 篇\n\n")

    for i, item in enumerate(all_articles, 1):
        article_data = item.get('data', {})
        title = article_data.get('title', '无标题')
        url = article_data.get('canonicalUrl', '无链接')
        publish_timestamp = article_data.get('publishTime', None)  # 获取时间戳

        publish_time_hk = "未知"
        if publish_timestamp:
            try:
                dt_utc = datetime.utcfromtimestamp(publish_timestamp)
                dt_hk = dt_utc + timedelta(hours=8)  # 转为香港时间
                publish_time_hk = dt_hk.strftime("%Y-%m-%d %H:%M:%S")  # 格式化
            except Exception as e:
                print(f"  - 时间戳转换失败: {publish_timestamp}, 错误: {e}")
                publish_time_hk = str(publish_timestamp)  # 如果转换失败，直接输出原时间戳
                
        print(f"正在处理第 {i} 篇文章: {title[:30]}...") # 打印标题前30个字符作为进度
        
        try:
            # 发送 GET 请求获取文章页面内容
            response = requests.get(url)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 提取正文 (找到所有 class 包含 "whitespace-pre-wrap break-words" 的 <p> 标签)
                paragraphs = soup.find_all('p', class_='whitespace-pre-wrap break-words')
                content = "\n\n".join([p.get_text(strip=True) for p in paragraphs])
                
                # 写入 Markdown 格式
                f.write(f"## [{i}. {title}]({url})\n\n")
                f.write(f"**发布时间（香港时间）**: {publish_time_hk}\n\n")  # 添加时间信息
                f.write(f"{content}\n\n---\n\n") # 使用 --- 作为文章分隔符
                
            else:
                print(f"  - 请求文章失败，状态码: {response.status_code}")
                # 即使请求失败，也写入标题和链接，标记错误
                f.write(f"## [{i}. {title}]({url})\n\n")
                f.write(f"**发布时间（香港时间）**: {publish_time_hk}\n\n")  # 添加时间信息
                f.write(f"[Error: Failed to fetch content, status code {response.status_code}]\n\n---\n\n")
                
        except Exception as e:
            print(f"  - 抓取文章时出错: {e}")
            # 即使出错，也写入标题和链接，标记错误
            f.write(f"## [{i}. {title}]({url})\n\n")
            f.write(f"**发布时间（香港时间）**: {publish_time_hk}\n\n")  # 添加时间信息
            f.write(f"[Error: {str(e)}]\n\n---\n\n")

print(f"\n--- Markdown 文件 '{output_filename}' 写入完成！---")

开始写入 Markdown 文件: hk01_scam_articles.md
正在处理第 1 篇文章: 一周110宗經Carousell網購騙案呃過千萬　有人賣$1...
正在处理第 2 篇文章: 網上情緣騙案｜扮女人氹落疊稱贈遺產　冒警二次詐騙　共呃440...
正在处理第 3 篇文章: 警反詐騙拘682人涉$6.2億　詐騙集團聘洗黑錢集團洗白$4...
正在处理第 4 篇文章: 新型詐騙？2男女稱大學玩遊戲求傳自拍照　網民點出破綻：係騙案...
正在处理第 5 篇文章: 緬北電騙白家案　白所成一審後病亡　白應蒼等4人伏法...
正在处理第 6 篇文章: 內地23歲男廚師冒充殲-16機師　與上百名女子戀愛圖呃錢被捕...
正在处理第 7 篇文章: 騙徒冒認廉署人員呃市民捲洗黑錢案索款　廉署籲提高警覺、勿轉賬...
正在处理第 8 篇文章: 越南女首富張美蘭案　政府擬出售其遊艇與名牌包　債還詐騙賠償金...
正在处理第 9 篇文章: 廉署破兩集團拘33人涉呃1.5億科技資助　揭數碼港職員收賄助...
正在处理第 10 篇文章: 宏福苑火｜涉AI製假捐款記錄向庫務署申請收據扣稅　中醫師被捕...
正在处理第 11 篇文章: 挑手筋剁手指　緬北果敢徐老發建14個詐騙園區一審判死刑緩刑兩...
正在处理第 12 篇文章: 台中神棍藉「雙修」騙色　還逼簽性愛同意書　完事後嘲諷過程沉悶...
正在处理第 13 篇文章: 「猜猜我是誰」集團呃27長者$226萬　警拘5男包括兩內地骨...
正在处理第 14 篇文章: 警聯青協成立防騙守護平台　周一鳴：去年約2500名大專生被呃...
正在处理第 15 篇文章: 涉兩社運案男子　近3年前保釋候判時參與電騙被捕　今再判囚45...
正在处理第 16 篇文章: 新加坡警方再向Meta發指令　擴大打擊Facebook冒充詐...
正在处理第 17 篇文章: 緬北明家11人伏法　外交部：加大力度打擊電詐網賭，鏟除賭詐毒...
正在处理第 18 篇文章: 緬北四大家族明家犯罪集團案　明國平、明珍珍等11人被執行死刑...
正在处理第 19 篇文章: 假破產服務誘騙百人借貸　呃$1000萬「手續費」　警瓦解團夥...
正在处理第 20 篇文章: 海關關員騙薪罪成判囚　放棄判刑上訴　等候定罪上訴期間取消保釋...
正在处理第 21 篇文章: 